# Auto-Join

データアナリストにとっての悩みのひとつである、アドホックな分析における表結合。

データソースの異なる複数システムからダンプされたデータや様々なオープンデータ、表計算ファイルなどに対して、キーとなるカラムのひな型が異なる場合、その表を結合するためには、分析の事前処理として、その変換の作業を行わなければなりません。

E.Zhuらは "Auto-join: Joining tables by leveraging transformations." (E.Zhu, Y.He, and S.Chaudhuri, 2017) のなかで、この変換処理を自動的に構築し、与えられた2つの表に対して等結合を行う`Auto-Join`という仕組みを提案しました。

`Auto-Join`のアルゴリズムは、大規模なデータに対しても効率的に適用され、高い確率で正確な結合が可能であることを実験で評価しています。

ここでは、同論文を解説しながら、Pythonで`Auto-Join`の実装を進めていきます。

#### Reference
[E. Zhu, Y. He, and S. Chaudhuri. Auto-join: Joining tables by leveraging transformations. PVLDB, 10(10):1034-1045, 2017.](https://www.microsoft.com/en-us/research/publication/auto-join-joining-tables-leveraging-transformations/)

![Figure 1](./images/figure1.png)
**Figure 1:**  
(left): US presidents and popular votes.   
(right): US presidents and job approval rating. The right table uses last-name, comma, first-name, with (year-of-birth and yearof-death).

[E. Zhu, Y. He, and S. Chaudhuri. Auto-join: Joining tables by leveraging transformations. PVLDB, 10(10):1034-1045, 2017.]

### Equi-Joinの限界

Auto-Joinが扱っている、解決したい問題は何なのか。データを見ると理解が早いですね。異なるソースのデータを扱う場合 `Figure 1` のように、表結合しようにも、フォーマットが異なることが多々あります。
データウェアハウスなどは、こうした問題が起こらないように、事前にETLを通じてデータクレンジングや前処理を行うですが、アドホックなデータ分析では、こういった前処理に多大な時間がかかってしまいます。

`Figure 1`:  
* 左: `GivenName [M. [M. ]]FamilyName`
* 右: `Familyname, Firstname[ M. M.](yyyy-[yyyy])`

このフォーマットの違いを修正する構文的変換を自動的に見つけることがこの論文のテーマです。


比較対象の先行研究として、`fuzzy join` という方法が挙げられており、これはトークン分割、距離関数、しきい値の組み合わせを元に、キーの組み合わせを探索するもののようです。しかし、この方法はあらゆるデータの型に合わせるにはあまりにも計算量が大きく、またその値を決めることはとても難しいと考えられます。

例えば、単語でトークン分割をする場合、

`Figure 1` の最後の行
* `Ronald Reagan`
* `Reagan, Ronald(1911-2004)`

この組のJaccard距離（類似度）は$0.66$となるため、しきい値は少なくともこれ以上大きな値にならなければなりません。

$$1.0-\frac{|v_s\cap v_t|}{|v_s\cup v_t|}=1.0-\frac{|\{Reagan\}|}{|\{Ronald, Regan, Ronald(1911-2004)\}|}=1.0-\frac{1}{3}=0.66$$

しかし、しきい値を$0.66$とした場合、

* `George W. Bush`
* `Bush, George H. W.(1924-)`

この組の類似度は$0.4$となり、これは正しい結果になりません。
$$1.0-\frac{|v_s\cap v_t|}{|v_s\cup v_t|}=1.0-\frac{|\{George,W,Bush\}|}{|\{George,W,Bush,H,(1924-)\}|}=1.0-\frac{3}{5}=0.4$$

しきい値に基づく、決定境界では常に正しくキーの組合せを見つけることが難しいことがわかります。

In [1]:
m = 'takahiro.hagino@bizreach.co.jp'
[m[c:len(m)] + '$' * c for c in range(0, len(m))]

['takahiro.hagino@bizreach.co.jp',
 'akahiro.hagino@bizreach.co.jp$',
 'kahiro.hagino@bizreach.co.jp$$',
 'ahiro.hagino@bizreach.co.jp$$$',
 'hiro.hagino@bizreach.co.jp$$$$',
 'iro.hagino@bizreach.co.jp$$$$$',
 'ro.hagino@bizreach.co.jp$$$$$$',
 'o.hagino@bizreach.co.jp$$$$$$$',
 '.hagino@bizreach.co.jp$$$$$$$$',
 'hagino@bizreach.co.jp$$$$$$$$$',
 'agino@bizreach.co.jp$$$$$$$$$$',
 'gino@bizreach.co.jp$$$$$$$$$$$',
 'ino@bizreach.co.jp$$$$$$$$$$$$',
 'no@bizreach.co.jp$$$$$$$$$$$$$',
 'o@bizreach.co.jp$$$$$$$$$$$$$$',
 '@bizreach.co.jp$$$$$$$$$$$$$$$',
 'bizreach.co.jp$$$$$$$$$$$$$$$$',
 'izreach.co.jp$$$$$$$$$$$$$$$$$',
 'zreach.co.jp$$$$$$$$$$$$$$$$$$',
 'reach.co.jp$$$$$$$$$$$$$$$$$$$',
 'each.co.jp$$$$$$$$$$$$$$$$$$$$',
 'ach.co.jp$$$$$$$$$$$$$$$$$$$$$',
 'ch.co.jp$$$$$$$$$$$$$$$$$$$$$$',
 'h.co.jp$$$$$$$$$$$$$$$$$$$$$$$',
 '.co.jp$$$$$$$$$$$$$$$$$$$$$$$$',
 'co.jp$$$$$$$$$$$$$$$$$$$$$$$$$',
 'o.jp$$$$$$$$$$$$$$$$$$$$$$$$$$',
 '.jp$$$$$$$$$$$$$$$$$$$$$$$$$$$',
 'jp$$$$$$$$$$$$$$$$

#### Solution Overview.
##### Step1: Find Joinable Row Pairs.
##### Step2: Learn Transformation.
##### Step3: Constrained Fuzzy Join.
|Symbol|Description|
|:-----|:----------|
|$T_s, T_t$|$T_s$ is the source table, $T_t$ is the target table.|
|$R_s, R_t$|A row in $T_s$ and $T_t$, respectively|
|$C_s, C_t$|A column in $T_s$ and $T_t$, respectively|
|$Q_q(v)$  |The q-grams of string value $v$|
|$Q_q(C_s)$|The multi-set of all q-grams ***of distinct values*** in Cs|
|$Q_q(C_t)$|The multi-set of all q-grams ***of distinct values*** in Ct|

**example**
$$Q_5(Database)=\{Datab,ataba,tabas,abase\}$$

### 3.1.1 1-to-1 q-Gram Match

* $p_q(k)$ : the *probability* mass function for a $q$-gram whose frequency ranks at position k among all $q$-grams
  * ある$q$-gram がすべての$q$-gram中の、$k$番目になる確率

$$
p_q(k)=\frac
{\frac{1}{k^{s_q}}}
{\sum_{z=1}^{N}\frac{1}{z^{s_q}}}$$

* $k$ : rank of a `q-gram` by frequency （頻出度ランク）
* $N$ : total number of `g-gram`
* $s_q$ : constant for a given $q$


$$\Omega = \{Split, Concat, Substring, Constant, SelectK\}$$

**Definition 1.**
*Transformation Join Problem:*
Given two tables $T_s, T_t,$ and a predefined set of operators $\Omega$, find a transformation $P=o_1\cdot o_2\cdot o_3\cdot ...o_n$, using operators $o_i \in \Omega$, such that $P(T_s)$ can equi-join with key columns of $T_t$.

* $P(T_s)$ equi-join with key columns of $T_t$
  * 分割、連結、切出しなどの操作を加えて、$T_s$を$T_t$に変換する操作$P(T)$を発見しましょう

**Proposition 1.**
Given two columns $C_s$ and $C_t$ from
tables $T_s$ and $T_t$ respectively, each with $N$ $q$-grams from an
alphabet of size $\left|\Sigma\right|$ that follow the power-law distribution
above. The probability that a $q$-gram appears exactly once
in both $C_s$ and $C_t$ by chance is bounded from above by the
following.

> TODO
$$\sum{|\Sigma|^q}{k=1}\big(\big)^2$$

* $Q_q(C)$ : the multi-set of all the $q$-grams of distinct values in column $C$
* $F_q(g, C)$ : the number of occurences of a $q$-gram $g \in Q_q(C)$. 
  * あるグラム$g$のカラム$C$における出現回数
* $v_s$ : the cell value at row $R_s$ column $C_s$ in $T_s$
* $v_t$ : the cell value at row $R_t$ column $C_t$ in $T_t$

**Definition 2.**
Let $g$ be a $q$\-gram with $g \in Q_q(v_s)$ and
$g \in Q_q(v_t)$. If $F_q(g, C_s) = 1$ and $F_q(g, C_t) = 1$, then $g$ is
a 1-to-1 $q$-gram match between row pair $R_s$ and $R_t$ with
respect to the pair of column $C_s$ and $C_t$.

### 3.1.2 General n-to-m q-Gram Match

**Definition 3.**
Let $g$ be a $q$-gram with $F(g, C_s) = n \geq 1$
and $F(g, C_t) = m \geq 1$, then $g$ is a n-to-m $q$-gram match for
corresponding rows with respect to the pair of column $C_s$
and $C_t$.

* $\frac{1}{n \cdot m}$ : quantify the “goodness” of the
match.

test all possible $q$-grams

$g^*$ as the best prefix of all possible suffixes of $v$

$$g^*=
\underset{
\forall_g \in \operatorname{Prefixes}(u), 
u \in \operatorname{Suffixes}(v)}
{\operatorname{arg max}} \frac{1}{nm}$$

> 最強の$q$-グラム$g^*$を決めよう

$n=F(g,C_s)\gt0$ and $m=F(g,C_t)\gt0$

**Proposition 2.**
*Let $g_u^q$ be a prefix of a suffix $u$ with length $q$. As the length increases by 1 and  $g_u^q$ extends at the end, the $\frac{1}{nm}$ score of the longer prefix $g_u^{q+1}$ is monotonically non-increasing, or $F(g_u^{q+1},C_s) \le F(g_u^q,C_s)$* and $F(g_u^{q+1},C_t) \le F(g_u^q,C_t)$.

> A proof of this can be found in Appendix C.

### 3.1.4 Putting it together: Find Joinable Rows

iterate through distinct value $v \in C_s$, and use
$\operatorname{OptimalQGram}$
to efficiently find the best $q$-gram match


* $\{g^*, score, R_s, R_t\} \leftarrow \operatorname{OptimalQGram}(v, C_t)$
  * この問題は$\frac{1}{nm}$を最大化するgを見つけることが命題であり、$g^*$のときのペアとなる行とそのスコア$score=\frac{1}{nm}$である。

Suffixインデックスのすべてに対して

$$\forall_u \in \operatorname{Suffixes}(v)$$
ベストな$q$を
$$q^* \leftarrow \operatorname{BinarySearchQ}(u, C_t)$$

* $\operatorname{BinarySearchQ}(u,C)$ performs the binary search of $q^*$ that finds $g_u^{q^*}$
  * あるCのなかで

In [8]:
# ['takahiro', 'takuya', 'shingo', 'tsuyoshi']
m='takahiro';[m[c:len(m)] + '$' * c for c in range(0, len(m))]

['takahiro',
 'akahiro$',
 'kahiro$$',
 'ahiro$$$',
 'hiro$$$$',
 'iro$$$$$',
 'ro$$$$$$',
 'o$$$$$$$']

In [12]:
m='takahiro'
u = [m[c:len(m)] for c in range(0, len(m))][2]
ct = ['takahiro', 'takuya', 'shingo', 'tsuyoshi']
def binary_search_q(u, ct):
    a = 3
    b = len(u) + 1
    while a < b:
        h = a + (b - a) / 2 # TODO ???
        rt = query_index(ct, u[1:h])
        if abs(rt) > 0:
            a = h + 1
        else:
            b = h
    return a - 1

'kahiro'

In [4]:
# TODO Mの型
# OptimalQGramfind best q-gram match
# how get KeyColumns() from T_t
"""
This function is based on 
- Algorithm 1 Find joinable row pairs
- Algorithm 3 Complete pseudo code for joinable row pair
ts:
    T_s subject table
    Dataframe
tt:
    T_t target table
    Dataframe
"""
def FindJoinableRowPairs(ts, tt):
    m = {}
    for cs in ts.columns:
        for ct in keycolumns(tt.columns):
            cs.apply(lambda v: optimal_qgram(v, ct))
            m.append()
    return m.groupby().sort_values('score')

# E. DETAILED PSEUDO CODE
## E.1 Find Joinable Row Pairs

* $\operatorname{KeyColumns}(T)$ : returns all the single columns that is part of a key column in the table $T$.
* $\operatorname{Suffixes}(v)$ : returns all suffixes of a value $v$.
* $\operatorname{QueryIndex}(C,g)$ : uses a suffix array index built for the column $C$, and returns a list of rows containing $g$.
* $\operatorname{MaxByScore}$ : returns the optimal row pairs with the hightest score.
* $q^*$ : leads to the highest score should results in the smallest possible non-zero number of matches in $C_t$.
  * マッチ数が少なければ$\frac{1}{mn}$は大きくなる
* $r_t$ : the matching rows in $C_t$ returned by $\operatorname{QueryIndex}$
  * calling $\operatorname{QueryIndex}(C_t,g_u^{q^*})$ results in $|r_t| \ge 1$, while calling $\operatorname{QueryIndex}(C_t,g_u^{q+1^*})$ results in $|r_t|=0$.

**TODO**

- search "suffix array index" in this paper.


$C_s$ is not needed in finding $g_u^{q^*}$
1. $n=|r_s|>0$
2. when $m=|r_t|=0$, the score $\frac{1}{nm}$ becomes undefined and thus the corresponding $q$ is infeasible.
Therefore, $q^*$ is only obtained at the conditions mentioned above, and is not dependent on $C_s$.

* $\operatorname{BinarySearchQ}(u,C)$ performs the binary search of $q^*$ that finds $g_u^{q^*}$ in $C$.
* 最適な$q$、つまりサフィックスに対してもっとも長いプレフィックスを見つけることのできる$q$を、バイナリサーチをかけることで見つけます
* $q\lt3$のときはパフォーマンスに影響が出るため、ブレイカーを設けている